# Development Scratch

This notebook contains ideas/scratch/notes for development

In [1]:
import os
import sys
import django
import datetime as datetime
import json
import pandas as pd
import requests

from collections import defaultdict
from distutils.util import strtobool


os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ['DJANGO_SETTINGS_MODULE'] = 'schedule_bot.settings'

django.setup()

from availabilities.models import *
from availabilities.utils import check_availabilities
from availabilities.utils import get_availabilities_for_participants
from events.finite_state_machine import UserEventMachine
from events.models import *
from communications.models import *
from events.utils import get_all_event_participants
from events.utils import string_to_date_time, create_event
from unittest.mock import Mock

## Tests Webhook

In [2]:
def send_webhook(user, msg, url):
    """
    """
    data = {
        'From': user,  # Example sender number
        'Body': msg,  # Example message body\
    }

    # Convert the data to JSON and make the POST request
    response = requests.post(
        url, 
        data=json.dumps(data), 
        headers={'Content-Type': 'application/json'}
    )
    
    return response

In [3]:
FEB_23_9AM = datetime.datetime(
    year=2024, 
    month=2, 
    day=3, 
    hour=9,
    tzinfo=datetime.timezone.utc
)

FEB_23_5PM = datetime.datetime(
    year=2024, 
    month=2, 
    day=23, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

FEB_23_2PM = datetime.datetime(
    year=2024, 
    month=2, 
    day=23, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_23_5PM = datetime.datetime(
    year=2024, 
    month=2, 
    day=23, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

FEB_24_2PM = datetime.datetime(
    year=2024, 
    month=2, 
    day=24, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_24_5PM = datetime.datetime(
    year=2024, 
    month=2, 
    day=24, 
    hour=17,
    tzinfo=datetime.timezone.utc
)


FEB_25_2PM = datetime.datetime(
    year=2024, 
    month=2, 
    day=25, 
    hour=14,
    tzinfo=datetime.timezone.utc
)

FEB_25_5PM = datetime.datetime(
    year=2024, 
    month=2, 
    day=25, 
    hour=17,
    tzinfo=datetime.timezone.utc
)

In [4]:
# Resets Data for Testing
Event.objects.all().delete()
UserEvent.objects.all().delete()
UserEventTime.objects.all().delete()
EventTime.objects.all().delete()
SuggestedDate.objects.all().delete()
DateAvailability.objects.all().delete()
Availability.objects.all().delete()

(0, {})

In [5]:
# Generates test data: Not available 2022/2/3 9-5PM
andrew_defaults = {
    "date_time_start": FEB_23_9AM,
    "date_time_end": FEB_23_5PM,
    "is_available": False
}

jessie_defaults = {
    "date_time_start": FEB_24_2PM,
    "date_time_end": FEB_24_5PM,
    "is_available": False
}

andrew, __ = User.objects.get_or_create(username="Andrew")
jessie, __ = User.objects.get_or_create(username="Jessie")
luna, __ = User.objects.get_or_create(username="Luna")

UserContactInfo.objects.update_or_create(user=andrew, defaults={"whatsapp_phone_number": "+16047627278"})
UserContactInfo.objects.update_or_create(user=jessie, defaults={"whatsapp_phone_number": "+17783889268"})
UserContactInfo.objects.update_or_create(user=luna, defaults={"whatsapp_phone_number": "+16041234567"})

availability, __ = DateAvailability.objects.update_or_create(
    user=andrew,
    category="hockey_team",
    defaults=andrew_defaults
)

availability, __ = DateAvailability.objects.update_or_create(
    user=jessie,
    category="social_stuff",
    defaults=jessie_defaults
)


# Generates proposed times
proposed_times = [
    (FEB_23_2PM, FEB_23_5PM),
    (FEB_24_2PM, FEB_24_5PM),
    (FEB_25_2PM, FEB_25_5PM),
]

# Generates event
create_event(
    host_user=jessie,
    event_name="RV and Park Day",
    invitees=[andrew, luna],
    proposed_times=proposed_times
)

event = Event.objects.get(
    name="RV and Park Day",
)

# Peeks
event_time_df = pd.DataFrame(UserEvent.objects.filter(event=event).values(
    "user__username", 
    "event__name",
    "state",
    "is_host",
    "is_required"
))
event_time_df

,user__username,event__name,state,is_host,is_required
0,Jessie,RV and Park Day,waiting_for_others,True,True
1,Andrew,RV and Park Day,no_communication,False,True
2,Luna,RV and Park Day,no_communication,False,True


In [7]:
# Sends initial text
user_event = UserEvent.objects.get(user=andrew, event=event)
user_event_machine = UserEventMachine(user_event)
user_event_machine.send_initial_text()

Andrew's new state: waiting_response


In [ ]:
# URL of your local Django webhook endpoint
url = 'http://127.0.0.1:8000/communications/twilio/webhook'
jessie_contact = "778-388-9268"
send_webhook(jessie_contact, "No", url)

In [ ]:
response = send_webhook(jessie_contact, "2022-2-17 4pm", url)
response

In [ ]:
pd.DataFrame(UserEvent.objects.values())

In [ ]:
send_webhook(jessie_contact, "Yes", url)

In [ ]:
# Peeks
pd.DataFrame(UserEvent.objects.values())

## Tests Event Initialization

In [ ]:
endpoint = "events/api/v1/event-initialization"

host_id = User.objects.get(username="Andrew").id
event_name = "Bowling Party"
event_times = [
    ("2/14/24 14:00", "02/14/24 18:00"),
    ("2/15/24 14:00", "02/15/24 18:00")
]
invitee_ids = [
    User.objects.get(username="Jessie").id,
    User.objects.get(username="Luna").id,
]
data = {
    "user_id": host_id,
    "event_name": event_name,
    "event_times": event_times,
    "invitee_ids": invitee_ids
}

def use_api(endpoint, data):
    """
    """
    url = "http://127.0.0.1:8000/" + endpoint
    # Convert the data to JSON and make the POST request
    response = requests.post(
        url, 
        data=json.dumps(data), 
        headers={'Content-Type': 'application/json'}
    )
    
    return response

response = use_api(endpoint, data)
response.json()

In [ ]:
from twilio.rest import Client

# Twilio credentials
account_sid="AC1a396906b5cd2edf4ea2aa5d544ca1da"
auth_token="3dad5da24df7096c2bf8ed2daf416960"

# Initialize Twilio client
client = Client(account_sid, auth_token)

# Your Twilio WhatsApp number and recipient's WhatsApp number
from_whatsapp_number = '+16593992076'
# from_whatsapp_number = 'whatsapp:+14155238886'
to_whatsapp_number = '+17783889268'

# Send the message
message = client.messages.create(
    body='This is a test message!',
    from_=from_whatsapp_number,
    to=to_whatsapp_number
)

print(f"Message SID: {message.sid}")

In [ ]:
# client = Client(account_sid, auth_token)

conversation = client.conversations.v1.conversations.create(friendly_name='My First Conversation')

print(conversation.sid)

In [ ]:
conversation = client.conversations.v1 \
                     .conversations('CH3e1a0d2cbd3c4f3b8e2dc461436b9d57') \
                     .fetch()

print(conversation.chat_service_sid)

In [ ]:
client = Client(account_sid, auth_token)

participant = client.conversations \
    .v1 \
    .conversations('CH3e1a0d2cbd3c4f3b8e2dc461436b9d57') \
    .participants \
    .create(
         messaging_binding_address='+16047627278',
         messaging_binding_proxy_address='+16593992076'
     )

In [ ]:
participant

In [ ]:
message = client.messages.create(
  from_='whatsapp:+14155238886',
  body='Your appointment is coming up on July 21 at 3PM',
  to='whatsapp:+16047627278'
)
